In [1]:
!pip install MTCNN

  Obtaining dependency information for MTCNN from https://files.pythonhosted.org/packages/09/d1/2a4269e387edb97484157b872fa8a1953b53dcafbe4842a1967f549ac5ea/mtcnn-0.1.1-py3-none-any.whl.metadata
  Obtaining dependency information for opencv-python>=4.1.0 from https://files.pythonhosted.org/packages/3f/a4/d2537f47fd7fcfba966bd806e3ec18e7ee1681056d4b0a9c8d983983e4d5/opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 3.3 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 5.9 MB/s eta 0:00:00m eta 0:00:010:00:01


In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet50
import numpy as np
import cv2
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
from mtcnn import MTCNN
import os


2024-09-07 14:44:24.702797: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-07 14:44:24.705882: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-07 14:44:24.747630: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-07 14:44:25.626603: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Initialize the face detector (MTCNN) and feature extractor (ResNet50)
detector = MTCNN()
model = resnet50(pretrained=True)
model.fc = nn.Identity()  # Replace classification layer with identity layer
model.eval()


/home/durgauser/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/durgauser/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [3]:
# Preprocessing pipeline for images
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [4]:
#!pip install kaggle

In [5]:
#!mkdir -p ~/.kaggle

In [6]:
#!kaggle datasets download -d jessicali9530/celeba-dataset

In [7]:
# Path to your small dataset of celebrity images
celeba_img_path = './celeb_dataset/img_align_celeba/img_align_celeba/'  # Replace with your small dataset directory

# Load image file paths from the small dataset
celeba_image_files = os.listdir(celeba_img_path)
celeba_image_paths = [os.path.join(celeba_img_path, img_file) for img_file in celeba_image_files]



In [8]:
#print(celeba_image_files)

In [9]:
print("Total number of files", len(celeba_image_files))

Total number of files 202599


In [10]:
# Function to extract face embeddings from an image
def extract_face_embedding(image_path):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Detect face in the image
    detection = detector.detect_faces(image_rgb)

    if len(detection) > 0:
        x, y, w, h = detection[0]['box']
        face = image_rgb[y:y+h, x:x+w]
        face = Image.fromarray(face)
        face_tensor = preprocess(face).unsqueeze(0)

        with torch.no_grad():
            embedding = model(face_tensor).numpy()

        return embedding
    else:
        print(f"No face detected in {image_path}")
        return None

# Function to compare the user image against a small set of celebrity images
def find_celebrity_look_alike(user_image_path, celeb_image_paths, top_n=3):
    user_embedding = extract_face_embedding(user_image_path)

    if user_embedding is None:
        return "No face detected in the user image."

    similarities = []

    # Compare with each celebrity image
    for celeb_path in celeb_image_paths:
        celeb_embedding = extract_face_embedding(celeb_path)
        if celeb_embedding is not None:
            similarity = cosine_similarity(user_embedding, celeb_embedding)[0][0]
            similarities.append((celeb_path, similarity))

    # Sort celebrities by similarity (highest first)
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)

    # Return the top N most similar celebrities
    return similarities[:top_n] if similarities else "No face detected in any celebrity images."


In [ ]:
# Step 1: Compute celebrity embeddings and save to a file

celeb_embeddings = {}
for celeb_path in celeba_image_paths:
    celeb_name = os.path.basename(celeb_path)
    embedding = extract_face_embedding(celeb_path)
    if embedding is not None:
        celeb_embeddings[celeb_name] = embedding


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
1/1 ━━━━━━━

In [ ]:
import pickle
# Save embeddings to a file (using pickle or numpy)
with open('celeb_embeddings.pkl', 'wb') as f:
    pickle.dump(celeb_embeddings, f)

print(f"Celebrity embeddings saved successfully!")